In [ ]:
#idk diference between cv and cv2. I think cv2 has class functionality
#import cv

In [6]:
#library needed on beaglebone
import cv2
import numpy as np
import time
#import matplotlib
import matplotlib.pyplot as plt

In [7]:
def get_480p(file):
    img=cv2.imread(file)
    img=cv2.resize(img,(640,480))
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img,gray_img
def get_cut_480p(file,xcut=[0,640],ycut=[0,480]):
    img=cv2.imread(file)
    img=cv2.resize(img,(640,480))
    img1=img[ycut[0]:ycut[1],xcut[0]:xcut[1]]
    gray_img=cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    return img,img1,gray_img
def get_360p(file):
    img=cv2.imread(file)
    img=cv2.resize(img,(320,240))
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img,gray_img
def display(img):
    try:
        b,g,r = cv2.split(img)
        img2 = cv2.merge([b,g,r])
        imgplot = plt.imshow(img)
        plt.show()
    except:    
        imgplot = plt.imshow(img,cmap='gray')
        plt.show()
def hough_trans(img,gray,can_min=50,can_max=350,vote=100,plot_canny=0,plot_img=0):
    t1 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t2 = time.time()
    lines = cv2.HoughLines(edges,1,np.pi/180,vote)
    t3 = time.time()
    print 'canny',t2 - t1
    print 'hough',t3 - t2
    if plot_canny==1:
                display(edges)
    #print lines
    if plot_img==1:
        try:
            for rho,theta in lines[0]:
                print rho,theta
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
            display(img)
        except:
            print('could not plot')
    else:
        try:
            return len(lines[0])
        except:
            return 0
def hough_transp(img,gray,can_min=150,can_max=300,vote=150,
                 minLineLength = 20,maxLineGap = 15,plot_canny=0,plot_img=0):
    #hough that uses less test points, less computations
    t1 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t2 = time.time()
    lines = cv2.HoughLinesP(edges,1,np.pi/180,vote,minLineLength,maxLineGap)
    t3 = time.time()
    print 'canny',t2 - t1
    print 'hough',t3 - t2
    if plot_canny==1:
        display(edges)
    #print lines
    if plot_img==1:
        try:
            for x1,y1,x2,y2 in lines[0]:
                cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
            display(img)
        except:
            print('could not plot')
    else:
        try:
            return len(lines[0])
        except:
            return 0
def hough_trans_blur(img,gray,can_min=50,can_max=350,vote=100,plot_canny=0):
    t1 = time.time()
    blur = cv2.blur(gray,(2,2))
    t2 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t3 = time.time()
    lines = cv2.HoughLines(edges,1,np.pi/180,vote)
    t4 = time.time()
    print 'blur', t2 - t1
    print 'canny',t3 - t2
    print 'hough',t4 - t3
    #print lines
    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    if plot_canny==1:
        display(edges)
    display(img)

In [84]:
def wh_det(img,x1=10,x2=630,y1=470,y2=0,xit=10,yit=-10,slope_en=1,er_max=3):    
    #create green pointers on sidewalk
    count=[]
    tot=0
    for x in range(x1,x2,xit):
        test=er_max#used to ignore noise in the image. if the values goes too low the system stops incrementing in that y direction
        for y in range(y1,y2,yit):
            bgr=img[y,x]
            if abs(int(bgr[1])-int(bgr[2]))<14 and ((int(bgr[0])*1.25)>(int(bgr[1])+int(bgr[2]))/2):#works
                tot+=1
                if test<er_max:
                    test+=1
                #draw where true
                for xn in range(-2,3):
                    for yn in range(-2,3):
                        img[y+yn,x+xn]=[0,200,255]
            else:
                test-=1
                if test<1:
                    break
        count.append(tot)
        tot=0
    #calculate slope,
    slopes=[]
    c1=yit/xit
    print len(count)
    if slope_en==1:
        slopes=[]
        for x1 in range(0,len(count)-1):
            slopes.append(float(count[x1]-count[x1+1])*c1)   
    #print results
    print count
    print slopes
    display(img)


In [93]:
img,gray=get_360p('debug_img9.jpg')
wh_det(img,x1=0,x2=320,y1=230,y2=30,xit=5, yit=-5,er_max=5)

64
[37, 39, 38, 38, 38, 38, 39, 38, 38, 39, 39, 39, 39, 39, 39, 38, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 37, 38, 38, 38, 38, 38, 37, 37, 38, 37, 37, 37, 37, 37, 36, 36, 36, 36]
[2.0, -1.0, -0.0, -0.0, -0.0, 1.0, -1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, 1.0, -1.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0]


In [110]:
def control_distance(slopes,xit,yit,gain,l1,side=1,dist=40):#side=1, right of sidewalk
	if side==1:
		#find sidewalk edge
		zeroend=len(l1)-1
		for x in range(len(l1)/2, len(l1)):
			if l1[x]==0:
				zeroend=x
				break
		#offset from slope, positive for turn right
		it=0
		tot=0
		for x in range(zeroend-5, zeroend-1):
			if slopes[x]!=0:
				tot+=slopes[x]
				it+=1
		if it==0:
			return 20
		slope=tot/it
		b=l1[zeroend]*yit-slope*zeroend*xit
		dist_to_sw=(-b)/slope
		print 'distance to sidewalk, '+str(dist_to_sw)+' pixels'
		offset=gain*(dist_to_sw-dist)
		if offset>20:
			return 20
		elif offset<-20:
			return -20
		else:
			return offset
	else:
		pass

In [112]:
l1=[37, 39, 38, 38, 38, 38, 39, 38, 38, 39, 39, 39, 39, 39, 39, 38, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 37, 38, 38, 38, 38, 38, 37, 37, 38, 37, 37, 37, 37, 37, 36, 36, 36, 36]
slopes=[2.0, -1.0, -0.0, -0.0, -0.0, 1.0, -1.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -1.0, -1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, 1.0, -1.0, -0.0, -0.0, -0.0, -0.0, -1.0, -0.0, -0.0, -0.0]
print control_distance(slopes,5,5,1,l1,dist=500)#,side=1,dist=40)

distance to sidewalk, 495.0 pixels
-5.0


In [51]:
for x in range(10,310,5):
        for y in range(235,200,5):
            bgr=img[y,x]
            if abs(int(bgr[1])-int(bgr[2]))<14 and ((bgr[0]*1.25)>(bgr[1]+bgr[2])/2):
                print '1'

In [12]:
img,gray=get_480p('pics/1.jpg')
wh_det(img,x1=5,x2=635,y1=475,y2=105,xit=5,yit=-5,er_max=5)

126
[74, 74, 71, 73, 73, 74, 68, 72, 73, 72, 74, 74, 74, 73, 74, 74, 74, 74, 73, 74, 70, 74, 73, 74, 74, 73, 74, 74, 74, 74, 74, 73, 74, 73, 73, 72, 72, 72, 72, 73, 73, 73, 74, 74, 72, 71, 70, 71, 70, 69, 70, 70, 70, 70, 70, 69, 68, 73, 70, 67, 73, 73, 72, 74, 73, 74, 74, 73, 72, 71, 71, 71, 74, 74, 72, 74, 73, 73, 73, 74, 74, 72, 74, 73, 72, 70, 73, 73, 73, 71, 35, 28, 23, 20, 18, 15, 16, 12, 11, 10, 7, 0, 0, 1, 2, 0, 0, 0, 2, 4, 1, 3, 1, 1, 0, 1, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0]
[0.0, 1.2, 0.0, 0.2, 0.0, -0.4, 0.2, -0.6, 0.0, -0.6, 0.2, -0.6, -0.2, 0.0, -0.2, -4.0, -3.2, 0.0, -0.2, 0.0, -0.2]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: RuntimeWarning: overflow encountered in ubyte_scalars


In [11]:
img,gray=get_480p('pics/2.jpg')
wh_det(img,x1=5,x2=635,y1=475,y2=105,xit=5,yit=-5,er_max=5)

126
[62, 62, 64, 62, 62, 61, 63, 63, 62, 59, 64, 61, 60, 61, 61, 61, 64, 64, 62, 64, 63, 63, 65, 66, 65, 65, 72, 71, 70, 71, 71, 71, 71, 72, 68, 68, 68, 68, 69, 69, 70, 69, 69, 73, 73, 70, 70, 71, 71, 72, 72, 72, 72, 73, 74, 74, 74, 74, 74, 74, 74, 74, 74, 71, 71, 71, 71, 65, 62, 60, 57, 54, 51, 47, 45, 44, 42, 39, 33, 30, 24, 23, 21, 20, 23, 20, 20, 16, 16, 13, 11, 8, 5, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-0.2, -0.4, 0.8, 0.8, 1.2, -0.6, 0.2, 0.4, 0.4, 0.0, -0.6, -3.4, -2.4, -2.6, -2.0, -2.2, 0.0, 0.0, 0.0, 0.0, 0.0]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: RuntimeWarning: overflow encountered in ubyte_scalars


In [44]:
#capture video
cap = cv2.VideoCapture(1)
while 1:
    ret, frame = cap.read()
    if ret==True:#check if cap.read() returned frame
        display(frame)
        #image recognition code here
        cv2.imwrite('pseye_3.jpg', frame)
    break

In [48]:
int(frame[10,10][0])

80

In [7]:
img,gray=get_480p('pseye_1.jpg')
wh_det(img,x1=30,x2=620,y1=470,y2=100,xit=10,yit=-10)

[23, 24, 23, 23, 23, 24, 24, 23, 23, 25, 24, 24, 23, 25, 26, 25, 24, 23, 24, 24, 25, 24, 24, 23, 23, 23, 23, 23, 25, 23, 23, 23, 23, 24, 23, 24, 23, 23, 23, 23, 23, 23, 23, 23, 23, 22, 22, 24, 24, 25, 22, 23, 22, 22, 22, 23, 22, 22, 23]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: RuntimeWarning: overflow encountered in ubyte_scalars


In [ ]:
#playing video
cap = cv2.VideoCapture('output.avi')

while(cap.isOpened()):
    ret, frame = cap.read()
    display(frame)
    break
cap.release()

In [4]:
#look at white concrete
img,reimg,gray=get_cut_480p('9.jpg',xcut=[400,470],ycut=[420,480])#get img, roi image, and gray cut img
#img,reimg,gray=get_cut_480p('1.jpg',xcut=[600,640],ycut=[400,480])#get img, roi image, and gray cut img
display(reimg)
b,g,r = cv2.split(reimg)
print reimg
#hsv to detect white?

[[[134 124 117]
  [107  97  90]
  [114 104  97]
  ..., 
  [111 102  95]
  [111 103  99]
  [113 104 100]]

 [[118 108 101]
  [112 102  95]
  [115 105  98]
  ..., 
  [125 116 109]
  [117 109 102]
  [118 110 103]]

 [[115 105  98]
  [117 107 100]
  [126 116 109]
  ..., 
  [116 108 101]
  [113 106  98]
  [122 114 107]]

 ..., 
 [[120 114 107]
  [122 116 111]
  [112 106 101]
  ..., 
  [104  96  90]
  [108  99  95]
  [105  96  92]]

 [[118 112 107]
  [123 117 112]
  [129 123 118]
  ..., 
  [111 102  97]
  [106  97  93]
  [106  97  93]]

 [[114 108 103]
  [119 113 108]
  [118 112 107]
  ..., 
  [109 101  95]
  [105  96  92]
  [106  97  93]]]


In [28]:
version=2
image='pseye_1'
#create green pointers on sidewalk
img,reimg,gray=get_cut_480p(image+'.jpg',xcut=[0,640],ycut=[0,480])#get img, roi image, and gray cut img
#display(reimg)
#b,g,r = cv2.split(reimg)
i=0;

#get from movie
#cap = cv2.VideoCapture('output.avi')
#ret, reimg = cap.read()
        
for x in range(0,640,10):
    for y in range(0,480,10):
        bgr=reimg[y,x]
        #also take the difference between g and r? if too high means its not white.
        if version==1:
            if bgr[0]*1.25>(bgr[1]+bgr[2])/2:#check if white with a bit more of blue    works
                i+=1
                #draw where true
                for x1 in range(-2,3):
                    for y1 in range(-2,3):
                        reimg[y+y1,x+x1]=[0,200,255]
        else:    
            if abs(int(bgr[1])-int(bgr[2]))<14 and ((bgr[0]*1.25)>(bgr[1]+bgr[2])/2):#works
                i+=1
                #draw where true
                for x1 in range(-2,3):
                    for y1 in range(-2,3):
                        reimg[y+y1,x+x1]=[0,200,255]
print i
#hsv to detect white?
display(reimg)
cv2.imwrite(image+'_white_detection_v'+str(version)+'.jpg', reimg)

1665


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:25: RuntimeWarning: overflow encountered in ubyte_scalars


True

In [ ]:
#debug white detection
#print reimg[350:480,220:380]
bgr=reimg[360,240]
print 1.2*bgr[1],bgr[1]
print abs(int(bgr[1])-int(bgr[2]))
if (abs(bgr[1]-bgr[2])<100) and ((bgr[0]*1.2)>(bgr[1]+bgr[2])/2):
    display(reimg[350:480,220:380])

In [ ]:
#find corners
img,gray=get_480p('1.jpg')

#gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

display(img)

In [ ]:
img1,gray=get_480p('2-knee.jpg')
display(img1)
#blur = cv2.bilateralFilter(gray,9,75,75)
#display(blur)
hough_trans(img1,gray,vote=220,can_min=150,can_max=250)

In [ ]:
#iterating hough
img,reimg,gray=get_cut_480p('1.jpg',xcut=[240,520],ycut=[200,480])#get img, roi image, and gray cut img
display(reimg)
#img1,gray=get_360p('2-knee.jpg')
it_vote=100
lvote=[it_vote]
lln=[]
mu=.75
n_lines=5
for x in range(0,5):
    ln=hough_trans(reimg,gray,vote=int(it_vote),can_min=200,can_max=250)
    lln.append(ln)
    fixed=n_lines-ln
    #iterate
    it_vote=it_vote-mu*fixed
    lvote.append(it_vote)
ln=hough_trans(reimg,gray,vote=int(it_vote),can_min=200,can_max=250,plot_canny=1,plot_img=1)
print(lvote)
plt.figure(1)
plt.plot(lvote)
plt.ylabel('Vote')
plt.xlabel('Iteration')
plt.figure(2)
plt.plot(lln)
plt.ylabel('Number of lines')
plt.xlabel('Iteration')
plt.show()

In [ ]:
img,gray=get_cut_480p('4-knee.jpg')
display(img)
it_vote=200
lvote=[it_vote]
lln=[]
mu=.45
n_lines=2
for x in range(0,20):
    ln=hough_trans(img1,gray,vote=int(it_vote),can_min=200,can_max=250)
    lln.append(ln)
    fixed=n_lines-ln
    #iterate
    it_vote=it_vote-mu*fixed
    lvote.append(it_vote)
ln=hough_trans(img1,gray,vote=int(it_vote),can_min=200,can_max=250,plot_canny=1,plot_img=1)
print(lvote)
plt.figure(1)
plt.plot(lvote)
plt.ylabel('Vote')
plt.xlabel('Iteration')
plt.figure(2)
plt.plot(lln)
plt.ylabel('Number of lines')
plt.xlabel('Iteration')
plt.show()

In [ ]:
#circles
img,gray=get_480p('1-a.jpg')
display(gray)
blur = cv2.medianBlur(gray,5)
display(blur)
circles = cv2.HoughCircles(blur,3,30,3)

circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(gray,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(gray,(i[0],i[1]),2,(0,0,255),3)
display(gray)

In [ ]:
#get low image resolution or take midsection of high res.
#blur, use the bit operator to only output where the sidewalk is, travel on right side.
#in forest area use the high intensity light as a sign of the sidewalk being there
#perhaps make algorithim to detect white

In [ ]:
#try to use the averaging filters to get the whole concrete slab
img,gray=get_480p('8.jpg')
blur = cv2.blur(gray,(2,2))
display(blur)
hough_trans(img,gray)

In [ ]:
#use bilateral filtering to remove texture(concrete) but is cpu intensive. 
img,gray=get_480p('8.jpg')
blur = cv2.bilateralFilter(gray,9,75,75)
display(blur)
hough_trans(img,gray)

In [ ]:
#individual colors
img,gray=get_480p('4-knee.jpg')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_blue = np.array([50,50,110])
upper_blue = np.array([255,255,130])
mask = cv2.inRange(hsv, lower_blue, upper_blue)
 # Bitwise-AND mask and original image
res = cv2.bitwise_and(img,img, mask= mask)
display(img)
display(mask)
display(res)

In [ ]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img=cv2.imread('4-knee.jpg')

In [ ]:
#returns (height, width, pixel element number)
height, width, pixel=img.shape
print img.shape

In [ ]:
imgplot = plt.imshow(img)
plt.show()

In [ ]:
img2 = img[280:340, 330:390]#take section of the picture
img[273:333, 100:160] = img2#put that section on original picutre

In [ ]:
imgplot = plt.imshow(img)
plt.show()

In [ ]:
b,g,r = cv2.split(img)#full splitting into colors
b = img[:,:,0]#individual blue color
imgplot = plt.imshow(b)
plt.show()

In [ ]:
#Grayscale
gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgplot = plt.imshow(gray_img)
plt.show()

In [ ]:
#HSV
hsv_img=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
imgplot = plt.imshow(hsv_img)
plt.show()

In [ ]:
print img[0,0]
print gray_img[0,0]
print hsv_img[0,0]

In [ ]:
#canny edge detection
img = cv2.imread('4-knee.jpg',0)
edges = cv2.Canny(img,100,200)

plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
img = cv2.imread('1-a.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,160,300,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180,200)
#print lines
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

plt.subplot(121),plt.imshow(edges,cmap = 'gray')
plt.subplot(122),plt.imshow(img,cmap = 'gray')
#plt.imshow(img,cmap='gray')
plt.show()
#cv2.imwrite('houghlines1.jpg',img)

In [ ]:
#hough that uses less test points, less computations
img = cv2.imread('4-knee.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
minLineLength = 100
maxLineGap = 20
lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
#print lines
for x1,y1,x2,y2 in lines[0]:
    cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
plt.subplot(121),plt.imshow(edges,cmap = 'gray')
plt.subplot(122),plt.imshow(img,cmap = 'gray')
plt.show()
#cv2.imwrite('houghlines2.jpg',img)